In [1]:
!pip install langchain-text-splitters
!pip install sentence-transformers

# load nghị định 100-2019 NĐ-CP

In [2]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [3]:
file_path = '/content/Nghị định-100-2019-NĐ-CP.docx.txt'
with open(file_path, 'r') as file:
            content = file.read()
            print(content)

﻿CHÍNH PHỦ
---------
Số: 100/2019/NĐ-CP
	CỘNG HÒA XÃ HỘI CHỦ NGHĨA VIỆT NAM
Độc lập - Tự do - Hạnh phúc
-------------------
Hà Nội, ngày 30 tháng 12 năm 2019
	



NGHỊ ĐỊNH
Quy định xử phạt vi phạm hành chính
trong lĩnh vực giao thông đường bộ và đường sắt
-------------


Căn cứ Luật Tổ chức Chính phủ ngày 19 tháng 6 năm 2015;
Căn cứ Luật Xử lý vi phạm hành chính ngày 20 tháng 6 năm 2012;
Căn cứ Luật Giao thông đường bộ ngày 13 tháng 11 năm 2008;
Căn cứ Luật Đường sắt ngày 16 tháng 6 năm 2017;
Theo đề nghị của Bộ trưởng Bộ Giao thông vận tải;
Chính phủ ban hành Nghị định quy định xử phạt vi phạm hành chính trong lĩnh vực giao thông đường bộ và đường sắt.


Chương I
NHỮNG QUY ĐỊNH CHUNG


Điều 1. Phạm vi điều chỉnh
1. Nghị định này quy định về hành vi vi phạm hành chính; hình thức, mức xử phạt, biện pháp khắc phục hậu quả đối với từng hành vi vi phạm hành chính; thẩm quyền lập biên bản, thẩm quyền xử phạt, mức phạt tiền cụ thể theo từng chức danh đối với hành vi vi phạm hành chính trong

# split content to chunk

In [4]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=512,
    chunk_overlap=50,
    length_function=len,
    is_separator_regex=False,
)

In [5]:
chunks_ = text_splitter.create_documents([content])
chunks = [c.page_content for c in chunks_]
print(f"The text has been broken down in {len(chunks)} chunks.")

The text has been broken down in 825 chunks.


# convert chunk to vector
I use dangvantuan/vietnamese-embedding model for embedding model.

In [6]:
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim

sentences = ['That is a happy person', 'That is a very happy person']

embedding_model = SentenceTransformer('dangvantuan/vietnamese-embedding').cuda() #('thenlper/gte-large').cuda()
embeddings = embedding_model.encode(sentences)
print(cos_sim(embeddings[0], embeddings[1]))

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tensor([[0.9351]])


In [7]:
embeddings.shape

(2, 768)

In [8]:
import torch
import numpy as np

embeddings_ = []
batch_size = 16

for _id in range(0, len(chunks)//batch_size + int(len(chunks) % batch_size != 0)):
    start_id = _id * batch_size
    end_id = min(len(chunks), start_id + batch_size)
    print(start_id, end_id)
    batch_chunks = chunks[start_id:end_id]
    with torch.no_grad():
        embeddings = embedding_model.encode(batch_chunks)
    embeddings_.append(embeddings)

embeddings = torch.tensor(np.concatenate(embeddings_, 0))
embeddings.shape

0 16
16 32
32 48
48 64
64 80
80 96
96 112
112 128
128 144
144 160
160 176
176 192
192 208
208 224
224 240
240 256
256 272
272 288
288 304
304 320
320 336
336 352
352 368
368 384
384 400
400 416
416 432
432 448
448 464
464 480
480 496
496 512
512 528
528 544
544 560
560 576
576 592
592 608
608 624
624 640
640 656
656 672
672 688
688 704
704 720
720 736
736 752
752 768
768 784
784 800
800 816
816 825


torch.Size([825, 768])

# load LLM model
I use Qwen 2 7B Instruct

In [9]:
from transformers import AutoModelForCausalLM, AutoTokenizer
device = "cuda" # the device to load the model onto

model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2-7B-Instruct",
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2-7B-Instruct")

config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/27.8k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

# convert question to embedding

In [11]:
question = 'vi phạm nồng độ cồn bị phạt bao nhiêu tiền?' #'uống rượu bia khi tham gia giao thông có bị phạt tiền không?'
query_embedding = embedding_model.encode([question])
query_embedding = torch.tensor(query_embedding)
query_embedding.shape

torch.Size([1, 768])

# calulate top 20 similar chunk by cosine

In [12]:
cos = torch.nn.CosineSimilarity(dim=1, eps=1e-6)
scores = cos(query_embedding.cuda(), embeddings.cuda())

In [13]:
indices = scores.topk(20).indices.cpu().long().numpy()
indices

array([650, 664, 665, 649, 102,  69,  75,  74, 134, 136, 151, 226, 262,
       252, 260, 314, 767, 418, 236, 648])

In [14]:
import numpy as np
chunks_ = np.array(chunks)
chunks_[indices]

array(['2. Phạt từ 4.000.000 đồng đến 6.000.000 đồng đối với hành vi khi làm nhiệm vụ mà trong máu hoặc hơi thở có nồng độ cồn vượt quá 50 miligam đến 80 miligam/100 mililít máu hoặc vượt quá 0,25 miligam đến 0,4 miligam/1 lít khí thở.\n3. Phạt từ 6.000.000 đồng đến 8.000.000 đồng đối với một trong các hành vi vi phạm sau đây:\na) Khi làm nhiệm vụ mà trong máu hoặc hơi thở có nồng độ cồn vượt quá 80 miligam/100 mililít máu hoặc vượt quá 0,4 miligam/1 lít khí thở;',
       'b) Khi làm nhiệm vụ mà trong máu hoặc hơi thở có nồng độ cồn nhưng chưa vượt quá 50 miligam/100 mililít máu hoặc chưa vượt quá 0,25 miligam/1 lít khí thở.\n6. Phạt tiền từ 16.000.000 đồng đến 18.000.000 đồng đối với hành vi khi làm nhiệm vụ mà trong máu hoặc hơi thở có nồng độ cồn vượt quá 50 miligam đến 80 miligam/100 mililít máu hoặc vượt quá 0,25 miligam đến 0,4 miligam/1 lít khí thở.\n7. Phạt tiền từ 30.000.000 đồng đến 40.000.000 đồng đối với một trong các hành vi vi phạm sau đây:',
       'a) Khi làm nhiệm vụ m

# create question prompt

In [15]:
Contexts = "\n".join(chunks_[indices].tolist())
print(Contexts)

2. Phạt từ 4.000.000 đồng đến 6.000.000 đồng đối với hành vi khi làm nhiệm vụ mà trong máu hoặc hơi thở có nồng độ cồn vượt quá 50 miligam đến 80 miligam/100 mililít máu hoặc vượt quá 0,25 miligam đến 0,4 miligam/1 lít khí thở.
3. Phạt từ 6.000.000 đồng đến 8.000.000 đồng đối với một trong các hành vi vi phạm sau đây:
a) Khi làm nhiệm vụ mà trong máu hoặc hơi thở có nồng độ cồn vượt quá 80 miligam/100 mililít máu hoặc vượt quá 0,4 miligam/1 lít khí thở;
b) Khi làm nhiệm vụ mà trong máu hoặc hơi thở có nồng độ cồn nhưng chưa vượt quá 50 miligam/100 mililít máu hoặc chưa vượt quá 0,25 miligam/1 lít khí thở.
6. Phạt tiền từ 16.000.000 đồng đến 18.000.000 đồng đối với hành vi khi làm nhiệm vụ mà trong máu hoặc hơi thở có nồng độ cồn vượt quá 50 miligam đến 80 miligam/100 mililít máu hoặc vượt quá 0,25 miligam đến 0,4 miligam/1 lít khí thở.
7. Phạt tiền từ 30.000.000 đồng đến 40.000.000 đồng đối với một trong các hành vi vi phạm sau đây:
a) Khi làm nhiệm vụ mà trong máu hoặc hơi thở có nồng

In [16]:
question_prompt = f'''Using the information contained in the context,
give a comprehensive answer to the question.
Respond only to the question asked, response should be concise and relevant to the question.
Provide the number of the source document when relevant.
If the answer cannot be deduced from the context, do not give an answer.

Context:
{Contexts}

Now here is the question you need to answer.

Question: {question}'''

In [17]:
print(question_prompt)

Using the information contained in the context,
give a comprehensive answer to the question.
Respond only to the question asked, response should be concise and relevant to the question.
Provide the number of the source document when relevant.
If the answer cannot be deduced from the context, do not give an answer.

Context:
2. Phạt từ 4.000.000 đồng đến 6.000.000 đồng đối với hành vi khi làm nhiệm vụ mà trong máu hoặc hơi thở có nồng độ cồn vượt quá 50 miligam đến 80 miligam/100 mililít máu hoặc vượt quá 0,25 miligam đến 0,4 miligam/1 lít khí thở.
3. Phạt từ 6.000.000 đồng đến 8.000.000 đồng đối với một trong các hành vi vi phạm sau đây:
a) Khi làm nhiệm vụ mà trong máu hoặc hơi thở có nồng độ cồn vượt quá 80 miligam/100 mililít máu hoặc vượt quá 0,4 miligam/1 lít khí thở;
b) Khi làm nhiệm vụ mà trong máu hoặc hơi thở có nồng độ cồn nhưng chưa vượt quá 50 miligam/100 mililít máu hoặc chưa vượt quá 0,25 miligam/1 lít khí thở.
6. Phạt tiền từ 16.000.000 đồng đến 18.000.000 đồng đối với h

# Ask Qwen2 7B

In [18]:
device = "cuda" # the device to load the model onto

messages = [
    {"role": "system", "content": "You are a helpful assistant. Please answer question more detail base on context!"},
    {"role": "user", "content": question_prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(device)

generated_ids = model.generate(
    model_inputs.input_ids,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(response)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Vi phạm với nồng độ cồn vượt quá 50 miligam đến 80 miligam/100 mililít máu hoặc vượt quá 0,25 miligam đến 0,4 miligam/1 lít khí thở thì bị phạt từ 4.000.000 đồng đến 6.000.000 đồng. Nếu nồng độ cồn vượt quá 80 miligam/100 mililít máu hoặc vượt quá 0,4 miligam/1 lít khí thở thì bị phạt từ 6.000.000 đồng đến 8.000.000 đồng.


# The result base on LLM model and embedding model.
## If we using Llama 3.1 405B, the answer is very good:

"Vi phạm nồng độ cồn có nhiều mức phạt khác nhau tùy thuộc vào mức độ vi phạm và loại phương tiện. Dưới đây là một số mức phạt cho người điều khiển xe ô tô và xe mô tô:

Điều khiển xe ô tô trên đường mà trong máu hoặc hơi thở có nồng độ cồn nhưng chưa vượt quá 50 miligam/100 mililít máu hoặc chưa vượt quá 0,25 miligam/1 lít khí thở: phạt tiền từ 6.000.000 đồng đến 8.000.000 đồng (điểm c khoản 8).
Điều khiển xe ô tô trên đường mà trong máu hoặc hơi thở có nồng độ cồn vượt quá 50 miligam đến 80 miligam/100 mililít máu hoặc vượt quá 0,25 miligam đến 0,4 miligam/1 lít khí thở: phạt tiền từ 10.000.000 đồng đến 12.000.000 đồng (điểm b khoản 10).
Điều khiển xe ô tô trên đường mà trong máu hoặc hơi thở có nồng độ cồn vượt quá 80 miligam/100 mililít máu hoặc vượt quá 0,4 miligam/1 lít khí thở: phạt tiền từ 30.000.000 đồng đến 40.000.000 đồng (điểm a khoản 10).
Điều khiển xe mô tô trên đường mà trong máu hoặc hơi thở có nồng độ cồn nhưng chưa vượt quá 50 miligam/100 mililít máu hoặc chưa vượt quá 0,25 miligam/1 lít khí thở: phạt tiền từ 2.000.000 đồng đến 4.000.000 đồng (khoản 1 Điều 63).
Điều khiển xe mô tô trên đường mà trong máu hoặc hơi thở có nồng độ cồn vượt quá 50 miligam đến 80 miligam/100 mililít máu hoặc vượt quá 0,25 miligam đến 0,4 miligam/1 lít khí thở: phạt tiền từ 4.000.000 đồng đến 6.000.000 đồng (khoản 2 Điều 63).
Điều khiển xe mô tô trên đường mà trong máu hoặc hơi thở có nồng độ cồn vượt quá 80 miligam/100 mililít máu hoặc vượt quá 0,4 miligam/1 lít khí thở: phạt tiền từ 6.000.000 đồng đến 8.000.000 đồng (điểm a khoản 3 Điều 63)."

## We need to rerank relevant chunk after get top 20 similar chunk to improve result

## I just use basic chunking strategy, maybe advanced chunking can help improve RAG QA system (late chunking, semantic chunking)

## Spelling errors in the text may occur when downloading as a .txt file. Perhaps we need to clean the data for better results.